In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pickle
import json
import seaborn as sns
import matplotlib.transforms as mtransforms
from bread import vis
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bread.algo import tracking
from bread.data import *
from sklearn.metrics import confusion_matrix
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
from glob import glob
import sys
import os.path
import datetime, json
from bread.algo.tracking import AssignmentDataset, GNNTracker, AssignmentClassifier, GraphLoader, accuracy_assignment, f1_assignment
from bread.data import *
from bread.algo.tracking import AssignmentDataset, GNNTracker, AssignmentClassifier, GraphLoader, accuracy_assignment, f1_assignment

from glob import glob
from pathlib import Path
import datetime, json, argparse
import torch, os
from pprint import pprint
from sklearn.metrics import confusion_matrix


from skorch.dataset import ValidSplit
from skorch.callbacks import LRScheduler, Checkpoint, ProgressBar, EpochScoring, WandbLogger, EarlyStopping

from importlib import import_module

import tqdm



## Load input assignment data

In [5]:
# /mlodata1/hokarami/fari/bread/data/generated/test_tracking/ass_graphs_original/colony003_segmentation__assgraph__dt_004__001_to_005.pt
assignment_graph_dir = Path("/mlodata1/hokarami/fari/bread/data/generated/test_tracking/ass_graphs_original")
# for colony000_segmentation__assgraph__dt_001__000_to_001.pt (example)
colony_name = "colony005"
time_diff = 3
first_frame = 113
next_frame = first_frame + time_diff

ass_graph_path = assignment_graph_dir/f'{colony_name}_segmentation__assgraph__dt_{time_diff:03d}__{first_frame:03d}_to_{next_frame:03d}.pt'
print(ass_graph_path)

/mlodata1/hokarami/fari/bread/data/generated/test_tracking/ass_graphs_original/colony005_segmentation__assgraph__dt_003__113_to_116.pt


## Load the saved model

In [6]:
model_folder = '/mlodata1/hokarami/fari/bread/best_models/27-10-trained-on-all'
model_dir = Path(model_folder)
algo = 'hungarian'
resultdir = Path(f'{model_dir}/results/{algo}_{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

with open(model_dir / 'hyperparams.json') as file:
    hparams = json.load(file)

net = AssignmentClassifier(
    GNNTracker,
    module__num_node_attr=hparams['num_node_attr'],
    module__num_edge_attr=hparams['num_edge_attr'],
    module__dropout_rate=hparams['dropout_rate'],
    module__encoder_hidden_channels=hparams['encoder_hidden_channels'],
    module__encoder_num_layers=hparams['encoder_num_layers'],
    module__conv_hidden_channels=hparams['conv_hidden_channels'],
    module__conv_num_layers=hparams['conv_num_layers'],
    module__num_classes=1,
    iterator_train=GraphLoader,
    iterator_valid=GraphLoader,
    criterion=torch.nn.BCEWithLogitsLoss,
).initialize()
net.load_params(model_dir / 'params.pt')
net.module_.train(False) 

GNNTracker(
  (node_encoder): MLP(12, 120, 120, 120, 120)
  (edge_encoder): MLP(10, 120, 120, 120, 120)
  (layers): ModuleList(
    (0-4): 5 x DeepGCNLayer(block=res+)
  )
  (out): MLP(120, 60, 30, 1)
)

In [7]:
graph = torch.load(ass_graph_path)
yhat = net.predict_assignment(graph, algo).flatten()
y = graph.y.squeeze().cpu().numpy()
y_raw = net.predict_raw(graph)
y_raw
# Calculate the percentage of assignment scores that are higher for each assigned cell
comparison_matrix = scores_copy > scores[np.newaxis, col_ind, :]
percentage_higher_scores = (np.sum(comparison_matrix, axis=0) / (scores.shape[0] - 1)) * 100


NameError: name 'scores_copy' is not defined